In [65]:
# This connects the python files to the main.js file
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import json

# Read the JavaScript data from the file
with open('main.js', 'r') as file:
    js_code = file.read()


# Extract the JavaScript array parts
start_index1 = js_code.find('[')
end_index1 = js_code.find(']', start_index1)
js_array_str1 = js_code[start_index1:end_index1 + 1]

start_index2 = js_code.find('[', end_index1)
end_index2 = js_code.find(']', start_index2)
js_array_str2 = js_code[start_index2:end_index2 + 1]

# Convert the JavaScript array strings to Python lists
numbers1 = json.loads(js_array_str1)
numbers2 = json.loads(js_array_str2)

print("Red Team:", numbers1)  # Output: [1, 2, 3, 4, 5]
print("Blue Team:", numbers2)  # Output: [6, 7, 8, 9, 10]


Red Team: ['Phoenix', 'Clove', 'Reyna', 'Skye', 'Cypher']
Blue Team: ['Clove', 'Sova', 'Reyna', 'Neon', 'Cypher']


In [66]:
# This calculates the best agent for a map given the user's match history
import requests

# Make a GET request to the API endpoint
response = requests.get('https://api.henrikdev.xyz/valorant/v3/matches/na/UF Fizz/f1zz')
data = ""

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Convert the response to JSON format
    data = response.json()
    # Now 'data' contains the JSON response from the API
    # print(data)
else:
    print('Failed to retrieve data from the API:', response.status_code)

# Step 1: Data Preparation
import pandas as pd

parsed_data = data

# Initialize a dictionary to store the best agent for each map
best_agents_for_maps = {}

# Loop through each match
for match in parsed_data['data']:
    #print(match)
    if match['metadata']['mode_id'] != 'competitive':
        pass
    # Extract map name
    map_name = match['metadata']['map']
    
    # Extract players and their characters
    players = match['players']['all_players']
    print(players)
    
    # Initialize a dictionary to store performance metrics for each character on the map
    character_performance = {}
    
    # Analyze performance of each character on the map
    #TODO: Separate performance into team vs enemy
    for player in players:
        character = player['character']
        if character not in character_performance:
            character_performance[character] = {'wins': 0, 'kills': 0, 'deaths': 0, 'assists': 0}
        
        # Update performance metrics
        if match['teams']['red']['has_won'] and player['team'] == 'Red':
            character_performance[character]['wins'] += 1
        character_performance[character]['kills'] += player['stats']['kills']
        character_performance[character]['deaths'] += player['stats']['deaths']
        character_performance[character]['assists'] += player['stats']['assists']
    
    # Determine the best character to play on the map based on performance metrics
    best_character = max(character_performance, key=lambda x: character_performance[x]['wins'])
    
    # Store the best agent for the map
    best_agents_for_maps[map_name] = best_character

# Print the best agent for each map
print("Best agent to play for each map:")
for map_name, best_agent in best_agents_for_maps.items():
    print(map_name + ":", best_agent)

[{'puuid': '536ec81b-3185-5024-97f2-1910800eb34d', 'name': 'Duke of Arrakis', 'tag': 'Tahah', 'team': 'Blue', 'level': 214, 'character': 'Clove', 'currenttier': 22, 'currenttier_patched': 'Ascendant 2', 'player_card': '7c2089bf-4294-df97-07cf-c4b38931342e', 'player_title': '2cb0c732-46b1-8c7f-81fc-25ac323f8962', 'party_id': 'a44fdf4c-4e7f-4203-8150-f76674ceed72', 'session_playtime': {'minutes': 3, 'seconds': 180, 'milliseconds': 180000}, 'behavior': {'afk_rounds': 0, 'friendly_fire': {'incoming': 0, 'outgoing': 0}, 'rounds_in_spawn': 0}, 'platform': {'type': 'PC', 'os': {'name': 'Windows', 'version': '10.0.19045.1.768.64bit'}}, 'ability_casts': {'c_cast': 6, 'q_cast': 9, 'e_cast': 49, 'x_cast': 3}, 'assets': {'card': {'small': 'https://media.valorant-api.com/playercards/7c2089bf-4294-df97-07cf-c4b38931342e/smallart.png', 'large': 'https://media.valorant-api.com/playercards/7c2089bf-4294-df97-07cf-c4b38931342e/largeart.png', 'wide': 'https://media.valorant-api.com/playercards/7c2089bf-4

In [77]:
import requests

def getPlayerStats(puuid, map, agent):
    maps = ['Ascent', 'Bind', 'Breeze', 'Icebox', 'Lotus', 'Split', 'Sunset']
    apiString = 'https://api.henrikdev.xyz/valorant/v1/by-puuid/lifetime/matches/na/' + puuid
    response = requests.get(apiString)
    data = ""

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Convert the response to JSON format
        data = response.json()
        # Now 'data' contains the JSON response from the API
        # print(data)
    else:
        print('Failed to retrieve data from the API:', response.status_code)

    parsed_data = data

    # Initialize a dictionary to store the best agent for each map
    player_stats = {}
    map_agent_win_count = [0, 0]
    kills, deaths = 0, 0
    
    # Loop through each match
    for match in parsed_data['data']:
        if match['meta']['mode'] == 'Competitive' and match['meta']['map']['name'] in maps:
            # Extract map name
            kills += int(match['stats']['kills'])
            deaths += int(match['stats']['deaths'])
            if(match['meta']['map']['name'] == map and match['stats']['character']['name'] == agent):
                team = match['stats']['team'].lower()
                if(team == 'blue'):
                    enemy = 'red'
                else:
                    enemy = 'blue'
                won_game = match['teams'][team] > match['teams'][enemy]
                if(won_game):
                    map_agent_win_count[0] += 1
                else:
                    map_agent_win_count[1] += 1

    player_stats['kd'] = kills / deaths
    if(sum(map_agent_win_count) > 0):
        player_stats['wr'] = map_agent_win_count[0] / (map_agent_win_count[1] + map_agent_win_count[0])
    else:
        player_stats['wr'] = 0.0
    return player_stats


# This calculates the win ratio of every agent on every map

# Make a GET request to the API endpoint
response = requests.get('https://api.henrikdev.xyz/valorant/v3/matches/na/angi/chi')
data = ""

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Convert the response to JSON format
    data = response.json()
else:
    print('Failed to retrieve data from the API:', response.status_code)

# Step 1: Data Preparation
import pandas as pd

parsed_data = data

# Initialize a dictionary to store win count and total match count for each agent on each map
puuid_list = []
map_name = ''
agent_list = []
# Loop through each match
players = parsed_data['data'][0]['players']['all_players']
map_name = parsed_data['data'][0]['metadata']['map']
    
# Update win count and total match count for each agent on the map
for player in players:
    puuid_list.append(player['puuid'])
    agent_list.append(player['character'])

for i in range(10):
    print(getPlayerStats(puuid_list[i], map_name, agent_list[0]))

{'kd': 1.0578313253012048, 'wr': 0.5}
{'kd': 1.079136690647482, 'wr': 0.0}
{'kd': 1.110419026047565, 'wr': 0.0}
{'kd': 1.1158798283261802, 'wr': 0.0}
{'kd': 1.0595854922279793, 'wr': 0.0}
{'kd': 0.9754281459419211, 'wr': 0.0}
{'kd': 0.9503105590062112, 'wr': 0.0}
{'kd': 0.7787418655097614, 'wr': 0.0}
{'kd': 0.9657794676806084, 'wr': 0.0}
{'kd': 0.9837728194726166, 'wr': 0.0}
